In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res

In [3]:
bounds = [(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,100),(0,100),(0,100),(0,1)]
X_optimal = [1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1]
y_optimal = -15
def G01(X):
    y = 5*(X[0]+X[1]+X[2]+X[3])-5*(X[0]**2+X[1]**2+X[2]**2+X[3]**2)-(X[4]+X[5]+X[6]+X[7]+X[8]+X[9]+X[10]+X[11]+X[12])
    g1 = 2*X[0]+2*X[1]+X[9]+X[10]-10
    g2 = 2*X[0]+2*X[2]+X[9]+X[11]-10
    g3 = 2*X[1]+2*X[2]+X[10]+X[11]-10
    g4 = -8*X[0]+X[9]
    g5 = -8*X[1]+X[10]
    g6 = -8*X[2]+X[11]
    g7 = -2*X[3]-X[4]+X[9]
    g8 = -2*X[5]-X[6]+X[10]
    g9 = -2*X[7]-X[8]+X[11]
    return y,g1,g2,g3,g4,g5,g6,g7,g8,g9

In [4]:
# g1 = lambda X: G01(X)[1]
# g2 = lambda X: G01(X)[2]
# g3 = lambda X: G01(X)[3]
# g4 = lambda X: G01(X)[4]
# g5 = lambda X: G01(X)[5]
# g6 = lambda X: G01(X)[6]
# g7 = lambda X: G01(X)[7]
# g8 = lambda X: G01(X)[8]
# g9 = lambda X: G01(X)[9]
# x0 = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,50,50,50,0.5]

In [5]:
# minfunction(g9,x0,bounds)

In [6]:
### 适应度函数
def fitnessG01_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    epsion = 1.5
    fmax = 5
    fmin = -305
    count = np.sum(np.array(G01(X))[1:10]>0)
    
    return -(G01(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG01_vio(X):
    """
    
    """
    fmax = 5
    fmin = -305
    vmax = 1182
    e = 0.1
    n=9
    
    res = G01(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:10]>0)
    
    for i in range(1,len(res)):
        if count > 0:
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
            
    return fitness  

def compare(X, X_optimal):
    """
    判断X和X_optimal之间的优劣性
    """
    res1 = G01(X)
    res2 = G01(X_optimal)
    
    flag = False
    
    amount_vio1 = np.sum(res1[1:13])
    amount_vio2 = np.sum(res2[1:13])
    
    if amount_vio1 == 0 and amount_vio2 != 0:
        flag = True
    if amount_vio1 == 0 and amount_vio2 == 0 and res1[0] < res2[0]:
        flag = True
    if amount_vio1 !=0 and amount_vio2 != 0 and amount_vio1 < amount_vio2:
        flag = True
            
    return flag

In [7]:
### PSO
# class PSO_com():
#     def __init__(self, pN, dim, max_iter, bounds):
#         self.wmax = 0.8 #最大的惯性权重
#         self.wmin = 0.4 #最小的惯性权重
#         self.c1 = 2  #加速因子
#         self.c2 = 2  #加速因子
#         self.pN = pN  # 粒子数量
#         self.dim = dim  # 搜索维度
#         self.max_iter = max_iter  # 迭代次数
#         self.bounds = bounds
        
#         self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
#         self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
#         self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
#         self.gbest = np.full(self.dim,0)      # 全局最佳位置：一个1*dim的矩阵
#         self.p_fit = np.full(self.pN,0.606)  # 每个个体的历史最佳适应值
#         self.fit = -10000                        # 全局最佳适应值
    
#     def init_Population(self):
#         """
#         初始化种群
#         :return:
#         """
#         for i in range(self.pN):
#             for j in range(self.dim):
#                 self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
#                 self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
#             self.pbest[i] = self.X[i]
#             if (compare(self.X[i],self.gbest)):
#                 self.gbest = self.X[i]
        
#     def iterator(self):
#         """
#         迭代
#         """
#         for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
#             for i in range(self.pN):
#                 self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest-self.X[i])
#                 self.X[i] = self.X[i] + self.V[i]
                
#                 """
#                 控制在搜索范围内
#                 """ 
#                 for j in range(self.dim):
#                     if self.X[i][j] < self.bounds[j][0]:
#                         self.X[i][j] = self.bounds[j][0]
#                     if self.X[i][j] > self.bounds[j][1]:
#                         self.X[i][j] = self.bounds[j][1]
                        
#             for i in range(self.pN):  # 更新gbest\pbest
#                 if (compare(self.X[i],self.pbest[i])):  # 更新个体最优
#                     self.pbest[i] = self.X[i]
#                     if (compare(self.pbest[i],self.gbest)):  # 更新全局最优
#                         self.gbest = self.pbest[i]
#         return self.gbest, self.fit

In [8]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10000.0)    # 全局最佳适应值
        self.gbest_1 = np.zeros((self.pN, self.dim)) # 第5000次迭代时的最优解
        self.g_fit_1 = np.full(self.pN,-10000.0) # 第5000次迭代时的最优适应度值
        self.gbest_2 = np.zeros((self.pN, self.dim)) # 第50000次迭代时的最优解
        self.g_fit_2 = np.full(self.pN,-10000.0) # 第50000次迭代时的最优适应度值
        self.gbest_3 = np.zeros((self.pN, self.dim)) # 第500000次迭代时的最优解
        self.g_fit_3 = np.full(self.pN,-10000.0) # 第500000次迭代时的最优适应度值
        self.FES = 500000   # 到达最优值需要的最小迭代次数
        self.devi = np.full(max_iter,0.0)  ##每次迭代完成最优解与实际最优解目标函数的偏差
        self.fe = 0
        self.su = 0
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG01_num(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
#             w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            w = 0.85
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG01_num(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/1500))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            if t == 5000-1:
                self.gbest_1 = self.gbest
                self.g_fit_1 = self.g_fit
            if t == 50000-1:
                self.gbest_2 = self.gbest
                self.g_fit_2 = self.g_fit
            if t == 500000-1:
                self.gbest_3 = self.gbest
                self.g_fit_3 = self.g_fit
            best_index = np.argmax(self.g_fit)
            best_solution = self.gbest[best_index]
            de = G01(best_solution)[0] - y_optimal
            self.devi[t] = de
            if np.sum(np.array(G01(best_solution)[1:10])>0) == 0:  ##
                self.fe = 1
            if de <= 0.0001:
                self.FES = t
                self.su = 1
            print(self.g_fit)  
        best_index = np.argmax(self.g_fit)
        best_solution = self.gbest[best_index]
        
        return self.gbest_1, self.g_fit_1, self.gbest_2, self.g_fit_2, self.gbest_3, self.g_fit_3, self.FES, self.devi, self.fe, self.su

In [9]:
def statistic(solutions,fitnesses):
    """
    给定25次的最优解和最优适应度值求统计值
    solutions：24次最优解
    fitnesses：24次最优适应度值
    """
    ress = []
    for i in range(len(fitnesses)):
        res = G01(solutions[i])[0]
        ress.append(res)
    mean = np.mean(ress)
    std = np.std(ress)
    
    best_index = np.argmax(fitnesses)
    best_solution = solutions[best_index]
    best_res = G01(best_solution)                         ##
    best_function = best_res[0] 
    best_vio_num = np.sum(np.array(best_res[1:10])>0)     ##

    worst_index = np.argmin(fitnesses)
    worst_solution = solutions[worst_index]
    worst_res = G01(worst_solution)                       ##
    worst_function = worst_res[0]
    worst_vio_num = np.sum(np.array(worst_res[1:10])>0)   ##

    median = np.median(fitnesses)
#     print(median)
    median_index = np.argwhere(fitnesses==median)[0][0]
#     print(median_index)
    median_solution = solutions[median_index]
    median_res = G01(median_solution)                      ###
    median_function = median_res[0]
    median_vio_num = np.sum(np.array(median_res[1:10])>0)  ###
    
    median_vio_vio_mean = 0.0
    flag = 0
    c1 = 0
    c2 = 0
    c3 = 0
    for i in range(1,10):                                  ###
        if median_res[i] > 0:
            flag = flag + 1
            median_vio_vio_mean = median_vio_vio_mean + median_res[i]
        if median_res[i] > 1.0:
            c1 = c1 + 1
        if median_res[i] > 0.01:
            c2 = c2 + 1
        if median_res[i] > 0.0001:
            c3 = c3 + 1
    if flag != 0:
        median_vio_vio_mean = median_vio_vio_mean / flag
    
    return best_function, best_vio_num, worst_function, worst_vio_num, median_function, median_vio_num, median_vio_vio_mean, c1, c2, c3, mean, std

In [10]:
def st(fes):
    best = np.min(fes)
    median = np.median(fes)
    worst = np.max(fes)
    mean = np.mean(fes)
    std = np.std(fes)
    
    return best, median, worst, mean, std

In [11]:
# solutions = [[0 for i in range(13)],
#              [0 for i in range(13)],
#              [1 for i in range(13)],
#              [0.2 for i in range(13)],
#              [0.3 for i in range(13)],
#              [0.4 for i in range(13)],
#              [0.5 for i in range(13)],
#              [0.6 for i in range(13)],
#              [0.7 for i in range(13)],
#              [0.8 for i in range(13)],
#              [0.9 for i in range(13)],
#              [0.11 for i in range(13)],
#              [0.12 for i in range(13)],
#              [0.13 for i in range(13)],
#              [0.14 for i in range(13)],
#              [0.21 for i in range(13)],
#              [0.13 for i in range(13)],
#              [0.14 for i in range(13)],
#              [0.21 for i in range(13)] 
# ]
# fitnesses = [0,0,0,0,0,0,0,0,0,0.1,0.2,0.2,0.2,0.3,0.3,0.4,0.5,0.6,0.7]
# statistic(solutions,fitnesses)

In [12]:
PSO = PSO_fit(pN=15000, dim=13, max_iter=500000, bounds=bounds)       ####

In [ ]:
solutions1 = []
fitnesses1 = []
solutions2 = []
fitnesses2 = []
solutions3 = []
fitnesses3 = []
fes = []
fs = 0        ## 可行率
sus = 0       ## 成功率

for i in range(24):
    
    PSO.init_Population()
    gbest_1, g_fit_1, gbest_2, g_fit_2, gbest_3, g_fit_3, FES, devi, fe, su = PSO.iterator()
    
    np.save("./results/G01/FES_3/gbest_" + str(i) + "_1.npy", gbest_1)      ###
    np.save("./results/G01/FES_3/g_fit_" + str(i) + "_1.npy", g_fit_1)      ###
    np.save("./results/G01/FES_4/gbest_" + str(i) + "_2.npy", gbest_2)      ###
    np.save("./results/G01/FES_4/g_fit_" + str(i) + "_2.npy", g_fit_2)      ###
    np.save("./results/G01/FES_5/gbest_" + str(i) + "_3.npy", gbest_3)      ###
    np.save("./results/G01/FES_5/g_fit_" + str(i) + "_3.npy", g_fit_3)      ###
    
    np.save("./results/G01/FE_" + str(i) + ".npy", FES)                    ###
    np.save("./results/G01/devi_" + str(i) + ".npy", devi)                   ###
    
    fes.append(FES)
    
    fs = fs + fe
    sus = sus + su
    
    best_index1 = np.argmax(g_fit_1)
    solutions1.append(gbest_1[best_index1])
    fitnesses1.append(g_fit_1[best_index1])
    
    best_index2 = np.argmax(g_fit_2)
    solutions2.append(gbest_2[best_index2])
    fitnesses2.append(g_fit_2[best_index2])
    
    best_index3 = np.argmax(g_fit_3)
    solutions3.append(gbest_3[best_index3])
    fitnesses3.append(g_fit_3[best_index3])

np.save("./results/G01/FES_3/solutions.npy", solutions1)     ###
np.save("./results/G01/FES_3/fitnesses.npy", fitnesses1)     ###

np.save("./results/G01/FES_4/solutions.npy", solutions2)     ###
np.save("./results/G01/FES_4/fitnesses.npy", fitnesses2)     ###

np.save("./results/G01/FES_5/solutions.npy", solutions3)     ###
np.save("./results/G01/FES_5/fitnesses.npy", fitnesses3)     ###

np.save("./results/G01/fes.npy", fes)                        ###

result0 = st(fes)
fs = fs / 24
sus = sus / 24

result1 = statistic(solutions1, fitnesses1)   
result2 = statistic(solutions2, fitnesses2)
result3 = statistic(solutions3, fitnesses3)

np.savetxt("results/G01/result0.txt", result0)
np.savetxt("results/G01/result1.txt", result1)
np.savetxt("results/G01/result2.txt", result2)
np.savetxt("results/G01/result3.txt", result3)
np.savetxt("results/G01/fs.txt", np.array(fs))
np.savetxt("results/G01/sus.txt", np.array(sus))

[-3048.29771655 -2287.28924436 -3051.02437445 ... -3086.69039043
 -1767.84923157 -1761.37554673]
[-1772.9849128  -1757.16249561 -1759.10978951 ... -3050.42474906
 -1756.27686765 -1755.11117043]
[-1754.26895882 -1751.97864731 -1756.7726716  ...     4.11246515
     2.14857751     1.95377128]
[1.         3.2934006  4.3849787  ... 4.11246515 4.88948747 7.65000667]
[4.6097744  5.6159009  5.78833307 ... 5.78403118 5.89858121 7.65000667]
[6.         5.63327326 6.         ... 6.         6.         7.65000667]
[6.         6.         6.         ... 6.12247084 6.         7.65000667]
[6.         6.         6.         ... 6.12247084 6.         7.65000667]
[6.         6.         6.         ... 6.12247084 8.23493556 7.65000667]
[6.94068015 6.         6.         ... 6.12247084 8.23493556 7.65000667]
[6.94068015 6.         6.         ... 8.66244206 8.23493556 7.65000667]
[8.25087862 6.         6.         ... 8.66244206 8.23493556 7.65000667]
[8.25087862 6.04340407 6.         ... 8.66244206 8.23493556 7

[14.95772421 14.95533838 14.97125811 ... 14.95986503 14.97305769
 14.9744179 ]
[14.95772421 14.95533838 14.97125811 ... 14.95986503 14.97305769
 14.9744179 ]
[14.95772421 14.97037322 14.97125811 ... 14.95986503 14.97305769
 14.9744179 ]
[14.95772421 14.97037322 14.97125811 ... 14.95986503 14.97305769
 14.9744179 ]
[14.95772421 14.97037322 14.97125811 ... 14.9703081  14.97305769
 14.9744179 ]
[14.95772421 14.97037322 14.97125811 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97156997 14.97037322 14.97125811 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97156997 14.97037322 14.97125811 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97156997 14.97037322 14.97125811 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97156997 14.97037322 14.98067712 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97983251 14.98031201 14.98067712 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97983251 14.98031201 14.98067712 ... 14.9703081  14.97305769
 14.9744179 ]
[14.97983251 14.98031201 14.98067712 ... 14.9703081 

[14.99727598 14.9981318  14.99729772 ... 14.99896249 14.99740666
 14.99866288]
[14.99727598 14.9981318  14.99729772 ... 14.99896249 14.99740666
 14.99866288]
[14.99727598 14.9981318  14.99729772 ... 14.99896249 14.99895461
 14.99866288]
